In [ ]:
!pip install --upgrade --force-reinstall "transformers[torch]" --no-deps

  Using cached transformers-4.57.6-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.57.6-py3-none-any.whl (12.0 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.6
    Uninstalling transformers-4.57.6:
      Successfully uninstalled transformers-4.57.6


In [ ]:
!pip install "accelerate"

In [ ]:
!pip install "numpy<2" --force-reinstall

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
pytensor 2.36.3 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatib

In [ ]:
#After Restart session
import langchain
import langchain_core
import langchain_openai
import langchain_community
from importlib.metadata import version
import transformers
import numpy as np

print(version("langchain"))
print(version("langchain-core"))
print(version("langchain-openai"))
print(version("langchain-community"))

print("langchain_v",langchain.__version__)
print("langchain_core",langchain_core.__version__)
print("langchain_community",langchain_community.__version__)
print("transformers",transformers.__version__)
print("numpy",np.__version__)


1.2.7
1.2.7
1.1.7
0.4.1
langchain_v 1.2.7
langchain_core 1.2.7
langchain_community 0.4.1
transformers 4.57.6
numpy 1.26.4


In [ ]:
!pip install -U langchain-huggingface

In [ ]:
from langchain_huggingface import HuggingFacePipeline
from langchain_community.agent_toolkits.load_tools import load_tools as lc_load_tools
from transformers import pipeline
from langchain_core.runnables import RunnableSequence

In [ ]:
hf_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_new_tokens=256,
    temperature=0.0,
)

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
!pip install wikipedia

In [ ]:
llm = HuggingFacePipeline(pipeline=hf_pipeline)
tools = lc_load_tools(["llm-math", "wikipedia"], llm=llm)

In [ ]:
for t in tools:
    print(t.name, "->", t.description)

Calculator -> Useful for when you need to answer questions about math.
wikipedia -> A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.


In [ ]:
def get_tool_by_keyword(keyword):
    for t in tools:
        if keyword.lower() in t.name.lower():
            return t
    return None


def agent(query: str):
    q = query.lower()

    # Math routing
    if any(x in q for x in ["*", "+", "-", "/", "calculate", "math"]):
        tool = get_tool_by_keyword("math") or get_tool_by_keyword("calc")
        if tool:
            return tool.run(query)

    # Wikipedia routing
    if any(x in q for x in ["who", "what", "where", "capital", "wiki"]):
        tool = get_tool_by_keyword("wiki")
        if tool:
            return tool.run(query)

    # Fallback
    return llm.invoke(query)

In [ ]:
agent("What is the capital of France?")
agent("Explain quantum computing")

'Quantum computing (QCC) is a technique that combines the properties of a quantum wave with the properties of a non-resonant wave. Quantum computing is a technique that combines the properties of a quantum wave with the properties of a nonresonant wave. Quantum computing is a technique that combines the properties of a quantum wave with the properties of a nonresonant wave. Quantum computing is a technique that combines the properties of a quantum wave with the properties of a nonresonant wave. Quantum computing is a technique that combines the properties of a quantum wave with the properties of a nonresonant wave. Quantum computing is a technique that combines the properties of a quantum wave with the properties of a nonresonant wave. Quantum computing is a technique that combines the properties of a quantum wave with the properties of a nonresonant wave. Quantum computing is a technique that combines the properties of a quantum wave with the properties of a nonresonant wave. Quantum 

In [ ]:
#Limited brief response as the model we used is very small
agent("paris!, thats interesting, any thoughts about paris, give me information related to source etc etc.")

'Paris is a city in France.'

In [ ]:
#Using a bigger Model & test the same steps as above or continue with base model for now
'''hf_pipeline = pipeline(
    "text-generation",
    model="tiiuae/falcon-7b-instruct",
    device=0,
    max_new_tokens=256,
    temperature=0.0,
)
llm_model = HuggingFacePipeline(pipeline=hf_pipeline)

tools = lc_load_tools(["llm-math", "wikipedia"], llm=llm_model)'''


'hf_pipeline = pipeline(\n    "text-generation",\n    model="tiiuae/falcon-7b-instruct",\n    device=0,\n    max_new_tokens=256,\n    temperature=0.0,\n)\nllm_model = HuggingFacePipeline(pipeline=hf_pipeline)\n\ntools = lc_load_tools(["llm-math", "wikipedia"], llm=llm_model)'

In [ ]:
#The steps here might fail as we are using newer version of Langchain which doesnt have
#create_react_agent, AgentExecutor etc..Thus commenting out
'''
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

# ReAct prompt
prompt = hub.pull("hwchase17/react")

# Agent
#agent = create_react_agent(llm=llm_model, tools=tools, prompt=prompt)
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
# Executor
executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

#If throws an error then Downgrade all your LangChain packages to a modern, compatible version that includes create_react_agent
'''

'\nfrom langchain.agents import create_react_agent, AgentExecutor\nfrom langchain import hub\n\n# ReAct prompt\nprompt = hub.pull("hwchase17/react")\n\n# Agent\n#agent = create_react_agent(llm=llm_model, tools=tools, prompt=prompt)\nagent = create_react_agent(llm=llm, tools=tools, prompt=prompt)\n# Executor\nexecutor = AgentExecutor(\n    agent=agent,\n    tools=tools,\n    verbose=True,\n    handle_parsing_errors=True\n)\n\n#If throws an error then Downgrade all your LangChain packages to a modern, compatible version that includes create_react_agent\n'

In [ ]:
'''question = "What is the capital of France and what is 17*23?"
result = executor.invoke({"input": question})
print(result["output"])
'''

'question = "What is the capital of France and what is 17*23?"\nresult = executor.invoke({"input": question})\nprint(result["output"])\n'

In [ ]:
#If versions werent downgraded , continue with this option
'''
def agent_router(query):
    if any(op in query for op in ["+", "-", "*", "/", "calculate"]):
        tool = next(t for t in tools if "Calculator" in t.name)
        return tool.run(query)
    elif any(k in query.lower() for k in ["who", "what", "where", "capital", "wiki"]):
        tool = next(t for t in tools if "Wikipedia" in t.name)
        return tool.run(query)
    else:
        return llm.invoke(query)

question = "What is the capital of France"
print(agent_router(question))
'''

'\ndef agent_router(query):\n    if any(op in query for op in ["+", "-", "*", "/", "calculate"]):\n        tool = next(t for t in tools if "Calculator" in t.name)\n        return tool.run(query)\n    elif any(k in query.lower() for k in ["who", "what", "where", "capital", "wiki"]):\n        tool = next(t for t in tools if "Wikipedia" in t.name)\n        return tool.run(query)\n    else:\n        return llm.invoke(query)\n\nquestion = "What is the capital of France"\nprint(agent_router(question))\n'

In [ ]:
#Two options,
#Downgrading & working with GPT model(older model say Gpt-4)
'''
!pip install -U \
  langchain==0.3.14 \
  langchain-core==0.3.32 \
  langchain-community==0.3.14 \
  langchain-openai==0.2.14 \
  pydantic==2.9.2
'''

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found

In [ ]:
#If downgraded
'''
#After restarting session
from langchain_openai import AzureChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor
from langchain_community.agent_toolkits.load_tools import load_tools
import os
'''

In [ ]:
#If not downgraded and if correctly using GPT-5x models
#Note** If using gpt model and AzureOpenAI or AzureChatOpenAI (refer: 'Working_with_AzureOpenAI' folder)
import os
from langchain_openai import AzureChatOpenAI
from langchain_community.agent_toolkits.load_tools import load_tools as lc_load_tools

from dotenv import load_dotenv
load_dotenv("/content/.env")

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("API_KEY"),
    deployment_name="gpt-5.1-chat",
    api_version="2024-12-01-preview"
)



In [ ]:
#from langchain_community.agent_toolkits.load_tools import load_tools as lc_load_tools
tools = lc_load_tools(["llm-math", "wikipedia"], llm=llm)

'''
#works with older versions of langchain
from langchain import hub
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

executor.invoke(
    {"input": "What is the capital of France"}
)
'''

#Getting ur custom functions
def get_tool_by_keyword(keyword):
    for t in tools:
        if keyword.lower() in t.name.lower():
            return t
    return None


def agent(query: str):
    q = query.lower()

    # Math routing
    if any(x in q for x in ["*", "+", "-", "/", "calculate", "math"]):
        tool = get_tool_by_keyword("math") or get_tool_by_keyword("calc")
        if tool:
            return tool.run(query)

    # Wikipedia routing
    if any(x in q for x in ["who", "what", "where", "capital", "wiki"]):
        tool = get_tool_by_keyword("wiki")
        if tool:
            return tool.run(query)

    # Fallback
    return llm.invoke(query)

In [ ]:
agent("Capital of France?")


'Page: List of capitals of France\nSummary: This is a chronological list of capitals of France. The capital of France has been Paris since its liberation in 1944.\n\n\n\nPage: Capital punishment in France\nSummary: Capital punishment in France (French: peine de mort en France) is banned by Article 66-1 of the Constitution of the French Republic, voted as a constitutional amendment by the Congress of the French Parliament on 19 February 2007 and simply stating "No one can be sentenced to the death penalty" (French: Nul ne peut être condamné à la peine de mort). The death penalty was already declared illegal on 9 October 1981 when President François Mitterrand signed a law prohibiting the judicial system from using it and commuting the sentences of the seven people on death row to life imprisonment. The last execution took place by guillotine, being the main legal method since the French Revolution; Hamida Djandoubi, a Tunisian citizen convicted of torture and murder on French soil, was 

In [ ]:
agent("Is it the tool or llm answering this question")

AIMessage(content='The answer you are reading right now is generated by the **LLM (the AI model)**, not an external tool.\n\nIf a tool is used, I will always indicate it clearly.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 16, 'total_tokens': 64, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5.1-chat-2025-11-13', 'system_fingerprint': None, 'id': 'chatcmpl-D1cviX5TUd4w6fpnORVwuqRUX25GP', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'con